# Imports

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding
)
from peft import PeftModel, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
import torch
import wandb

from utils import tokenize_dataset_for_qna
from prompt_templates import domain_bound_qna_prompt_template as prompt_template

# Configs

In [ ]:
data_path = "../data/domain_bound_data/"
train_data_path = data_path + "train.csv"
val_data_path = data_path + "val.csv"

max_len = 512

base_model_path = "../models/phi_qna_finetuned_attempt_4/final_pretrained"

model_id = "microsoft/Phi-3.5-mini-instruct"

model_output_dir = "../models/phi_domain_bound_qna_finetuned_attempt_1"

# Hyperparameters

In [ ]:
lora_r = 32
lora_alpha = 64
lora_target_modules = ["q_proj", "v_proj", "k_proj", "o_proj"]
batch_size = 32
quantization = None
lora_dropout = 0.05
epochs = 5
learning_rate = 5e-5

# Dataset

In [ ]:
print(prompt_template)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [ ]:
val_df = pd.read_csv(val_data_path)
train_df = pd.read_csv(train_data_path)

val_set = tokenize_dataset_for_qna(tokenizer, val_df, prompt_template, max_len)
train_set = tokenize_dataset_for_qna(tokenizer, train_df, prompt_template, max_len)

In [ ]:
train_set[0]

In [ ]:
train_set.save_to_disk(data_path + "tokenized")
# train_set = load_from_disk(data_path + "tokenized")

In [ ]:
wandb.init(
    project="med-domain_bound_qna_finetune",
    name="attempt_1",
    config={
        "model": model_id,
        "lora_r": lora_r,
        "lora_alpha": lora_alpha,
        "batch_size": batch_size,
        "epochs": epochs,
        "quantization": quantization,
        "lora_target_modules": lora_target_modules
    }
)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=quantization=="4bit",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

# Train

In [ ]:
training_args = TrainingArguments(
    output_dir=model_output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    eval_strategy="epoch",  # ✅ eval at each epoch
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=learning_rate,
    warmup_steps=500,
    fp16=True,
    report_to="wandb",
    run_name="attempt_1",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer, padding=False)
)

In [ ]:
trainer.train()

trainer.save_model(model_output_dir + "/final")
tokenizer.save_pretrained(model_output_dir + "/final")

## Merge model with lora weights

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

finetuned_model = PeftModel.from_pretrained(base_model, output_model_dir + "/final")
merged_model = finetuned_model.merge_and_unload()

In [ ]:
model.save_pretrained(output_model_dir +  "/final_pretrained")